# Customer Churn Prediction

## Introduction <a name="introduction"></a>

Customer churn, often referred to as churn rate, represents the scenario where a customer discontinues their relationship with a company, quit using a particular service, or no longer purchases a specific product. It is a significant business metric, primarily because the acquisition of new customers entails considerably higher costs compared to retaining existing ones.

Predicting customer churn is a strategic tool that empowers businesses to proactively identify potential risks associated with customer churn, allowing for timely intervention. By identifying these risks, they can take actions such as personalized marketing campaigns, extending offers, or improving customer service to ensure their continued loyalty and engagement with the company.

Customer churn prediction is a application that relies heavily on machine learning algorithms like logistic regression and random forests. While these ML techniques good at identifying customers likely to churn, statistical tools such as survival analysis have appeared as important components of this predictive process. While algorithms like logistic regression prediction offer insights into which customers are more prone to leaving, survival analysis goes a step further by not only predicting the likelihood of customer churn but also predicting when such events might occur. Hence, some argue that the survival analysis is essential for achieving more accurate and actionable insights in customer retention strategies. See : https://towardsdatascience.com/better-churn-prediction-part-2-5a1086fd3f51

In this kernel, I want to explore machine learning and survival analysis techniques to construct a robust customer churn prediction model. By doing so, I can evaluate their respective performance and determine which models show better predictive capabilities in tackling the complex challenge of customer churn. Following are the model I will be using, 

* Machine Learning Based
    * Logistic Regression
    * Random Forest

* Survival Models
    * COX
    * Survival Forest


I choose the Telco Customer Churn data stet available in Kaggle. The data set includes information about:

* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

### Load the libraries and the data

In [ ]:
# Common libraries for data analysis
import numpy as np
import pandas as pd

# Libraries for graphs
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix, mutual_info_score, recall_score, accuracy_score, precision_score, f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance

from statsmodels.stats.outliers_influence import variance_inflation_factor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# load the data
data=pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
data.head(3)

In [ ]:
# shape of the data
data.shape

In [ ]:
# type of each column
data.dtypes

In [ ]:
# unique values of each column
for col in data:
    print(col, ":" ,data[col].unique())

## Data Preprocessing <a name="Preprocessing"></a>

In [ ]:
# check for duplicate customers
data['customerID'].nunique()

In [ ]:
# Total charge appers as a string features that contains empty strings, so let's remove the rows with empty strings
data_df = data.loc[data['TotalCharges']!=' ']

In [ ]:
# remove the customer ID
data_df=data_df.drop(['customerID'],axis=1)

# convert TotalCharges to float
data_df = data_df.astype({"TotalCharges": float})

In [ ]:
# check for other missing value percentage
(data_df.isnull().sum() / len(data_df)).sort_values(ascending=False) * 100

## Exploratory Data Analysis <a name="EDA"></a>

In [ ]:
# check the class balance
print("Value counts of", data['Churn'].value_counts())
sns.histplot(data=data_df, x="Churn", hue="Churn", palette = "Set1")
plt.title('Churn of Customers')

As the plot describes, many of the data points belong to the Customers that have not left the organization which is around 73%. We have to keep in mind this 'Class Imbalance' when selecting training data.

In [ ]:
# gender based churn
sns.histplot(binwidth=1,
            x='gender',
            hue='Churn',
            data=data_df,
            stat="count",
            multiple="dodge", palette = "Set1")

In [ ]:
data_df['gender'].value_counts()

There is no significant differences in the gender for each two classes. The female and male Churn values are closer.

In [ ]:
# relationship between the tenure and the average total charges
ave_tot_ch=data_df.groupby(['tenure', 'Churn'])['TotalCharges'].mean().reset_index(name='AveTotalCharges')
sns.scatterplot(data=ave_tot_ch, x= 'tenure', y='AveTotalCharges', hue='Churn')

In [ ]:
# relationship between the tenure and the average total charges
ave_monthly_ch=data_df.groupby(['tenure', 'Churn'])['MonthlyCharges'].mean().reset_index(name='AverageMonthlyCharges')
sns.scatterplot(data=ave_monthly_ch, x= 'tenure', y='AverageMonthlyCharges', hue='Churn')

The relationship between the average monthly charges vs. tenure is a linear relationship. The plot indicates the higher the tenure, there is an increase in the average monthly charges. It also indicates that the customers who have churned paid a higher amount for the monthly charges than the customers who have not churned yet. We observed a similar trend in the previous plot, which shows the relationship between total charges vs. tenure with regard to churn.

In [ ]:
# relationship between the tenure and the average MonthlyCharges based on gender
ave_monthly_gen=data_df.groupby(['tenure', 'gender'])['MonthlyCharges'].mean().reset_index(name='AverageMonthlyCharges')
sns.scatterplot(data=ave_monthly_gen, x= 'tenure', y='AverageMonthlyCharges', hue='gender', palette = "Set1")

First, I will convert some binary categorical variables to binary values. Later, I will use pd.get_dummies() to apply one hot encoding to the rest of the categorical variables. 

In [ ]:
# Convert Yes and No values to 0 and 1
data_df['Churn']=data_df.Churn.map(dict(Yes=1, No=0))
#data_df['gender']=data_df.Partner.map(dict(Male=1, Female=0))
data_df['Partner']=data_df.Partner.map(dict(Yes=1, No=0))
data_df['Dependents']=data_df.Dependents.map(dict(Yes=1, No=0))
data_df['PhoneService']=data_df.PhoneService.map(dict(Yes=1, No=0))
data_df['PaperlessBilling']=data_df.PaperlessBilling.map(dict(Yes=1, No=0))

In [ ]:
data_df.head(3)

In [ ]:
data_df.dtypes

### Correlation Analysis

Correlation Analysis measures the correlation between the variables or features. In general, we expect no correlation among the independent variables and some correlation between the dependent variable and each independent variable. 

In [ ]:
# check correlation between the target and the features
conti_df= data_df.select_dtypes(exclude=[object])

corr_with_tot_count = conti_df.corr()["Churn"].sort_values(ascending=False)

plt.figure(figsize=(8,6))
corr_with_tot_count.drop("Churn").plot.bar()
plt.show()

In [ ]:
# check for correlation among the features

#pd.set_option('precision',2)
plt.figure(figsize=(6,6))

sns.heatmap(conti_df.drop(['Churn'],axis=1).corr(), square=True, annot=True,)
plt.suptitle("Pearson Correlation Heatmap")
plt.show()

In [ ]:
# create correlation matrix
corr_matrix = conti_df.drop(['Churn'],axis=1).corr().abs()

# select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# select one feature from the highly correlated features, threshold 0.7
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]

print(to_drop)

In [ ]:
# Based on the correlation analysis I will drop the following features from the data
df=data_df.drop(['TotalCharges', 'PhoneService'],axis=1)
df.head(3)

### Mutual Information

Mutual information (MI) measures the degree of dependence or statistical association between two random variables. In machine learning, it is  used for feature selection, feature engineering, and measuring the dependence between variables. We can use this to determine the degree of dependency between the dependent variable and each of the independent variables. Mutual information is a non-negative value. A higher mutual information indicates a stronger relationship or dependency between the variables, while a lower value suggests less dependency. 

By looking at the MI scores we can eliminate the independent variables that have a lower degree of dependence on the target variable.  

In [ ]:
# returns the MI scores
def compute_mutual_information(categorical_serie):
    return mutual_info_score(categorical_serie, df.Churn)

# select categorial variables excluding Churn
cat_vars = df.select_dtypes(include=object)

# compute the MI score between each categorical variable and the target
feature_importance_df = cat_vars.apply(compute_mutual_information).sort_values(ascending=False)

# visualize feature importance
plt.bar(feature_importance_df.index, height = feature_importance_df.values)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# drop the least important features that has low MI scores, the threshold hee is arbitarary
print(feature_importance_df)
todrop = feature_importance_df[feature_importance_df < 0.06]
todrop.index.values

In [ ]:
selected_features=df.drop(todrop.index.values,axis=1)
selected_features.head(3)

In [ ]:
selected_features.shape

### Variance Inflation Factor


Variance Inflation Factor (VIF) is used to detect the multicollinearity among independent variables within a regression model. VIF quantifies the degree to which the variance of estimated coefficients is inflated due to multicollinearity. This inflation is evaluated with respect to R-squared.

* VIF values above 4 indicate a potential issue that should be explored further.
* When VIF values are higher than 10, it indicates a more severe case of multicollinearity that requires corrections.

In [ ]:
def cal_VIF(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] =  X.columns
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                              for i in range(len(X.columns))] 
    return vif_data

# convert all the categorical data to numerical applying one hot encoding
input_df = pd.get_dummies(selected_features.drop(['Churn'],axis=1), dtype=float, drop_first = True)
cal_VIF(input_df)

In [ ]:
# based on the higher VIF, I will drop the following features from the data
input_df_new = pd.get_dummies(selected_features.drop(['Churn', 'OnlineSecurity', 'TechSupport'],axis=1), dtype=float, drop_first = True)
cal_VIF(input_df_new)

In [ ]:
# select the final set of data and seperate the target and independent variables to y and X
all_features_df = pd.get_dummies(selected_features[['SeniorCitizen', 'Partner', 'Dependents', 'PaperlessBilling','MonthlyCharges', 'Contract','tenure', 'Churn']],dtype=int, drop_first=True)
X = all_features_df.drop(['Churn'],axis=1)
y = all_features_df['Churn']

In [ ]:
X.head(3)

#### Scaling the data & seperating the train/test data

In [ ]:
# create the standard scaler 
scaler = StandardScaler()
# transform data
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns= X.columns)
X_scaled.head(3)

In [ ]:
# Train/ Test Split using stratify sampling to solve the class imbalance problem
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify = y, test_size=0.2, random_state=33)

print(f'Percentage of churn in train: {round(100 * (y_train[y_train == 1].shape[0] / len(y_train)),2)} %')
print(f'Percentage of churn in test: {round(100 * (y_test[y_test == 1].shape[0] / len(y_test)),2)} %')

## Logistic Regression

The first model I will use is the Logistic Regression model. It is a popular supervised learning algorithm mainly applied to binary classification problems. This model calculates the probability of a customer belonging to one of two classes, with outcomes constrained to the range of $[0, 1]$. Logistic regression extends the principles of linear regression but tailors them to probabilistically assess relationships between the target variable, "Churn," and other features. By doing so, it classifies customers into appropriate categories based on these calculated probabilities.

In [ ]:
# fit the model and predict
model_lr = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = model_lr.predict(X_test)

In [ ]:
# model_lr.predict_proba(X_test)

In [ ]:
# returns the model performance in terms of accuracy, classification report and the confusion matrix
def model_performance(y_test,y_pred, method):
    
    # calculate the accruacy of the model
    print("Accuracy score of the model", accuracy_score(y_test,y_pred))
    print("Classification report \n")
    
    #generate the classification report
    print(classification_report(y_test,y_pred))
    
    #generate the confusion matrix
    fig = plt.figure(figsize = (4,4))
    ax = fig.gca()
    cnf_matrix_log = confusion_matrix(y_test, y_pred)
    sns.heatmap(pd.DataFrame(cnf_matrix_log), annot=True,cmap="Reds" , fmt='g')
    ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title('Confusion matrix: '+  method + '\n', y=1.1)
    

In [ ]:
# lets measure the model performance
model_performance(y_test,y_pred, 'Logistic Regression')

The logistic regression model has attained 80% accuracy. There are more False negatives than False positives indicating high precision and low recall for overall model. For this context, I think we would be aiming for a fewer false positive, so that we can avoid a potential customer churn.

In [ ]:
    labels=['Churn','Not Churn']
    p = model_lr.predict_proba(X_test)
    if len(model_lr.classes_)!=2:
        raise ValueError('A binary class problem is required')
    if model_lr.classes_[1] == 1:
        pos_p = p[:,1]
    elif model_lr.classes_[0] == 1:
        pos_p = p[:,0]
    
    prob_df = pd.DataFrame({'probPos':pos_p, 'target': y_test})

    plt.hist(prob_df[prob_df.target==1].probPos, density=True, bins=25,alpha=.5, color='green',  label=labels[0])
    plt.hist(prob_df[prob_df.target==0].probPos, density=True, bins=25,alpha=.5, color='red', label=labels[1])
    plt.axvline(.5, color='blue', linestyle='--', label='Boundary')
    plt.xlim([0,1])
    plt.title('Distributions of Predictions', size=13)
    plt.xlabel('Positive Probability (predicted)', size=10)
    plt.ylabel('Samples (normalized scale)', size=10)
    plt.legend(loc="upper right")

In [ ]:
# apply cross validation to generate more generic result
cv_results = cross_validate(model_lr, X_train, y_train, cv=5)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy for Logistic Regression is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}")

In [ ]:
# apply the grid search for hyper parameter tunning 
grid={"C":np.logspace(-3,3,7), "penalty":["l2"]}
logreg_cv=GridSearchCV(model_lr,grid,cv=10)
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

## Random Forests

Next, I am using the Random Forest model which is an ensemble technique that depends on a collection of decision trees for making predictions. It operates as a non-parametric model, making no presumptions about data distributions. This algorithm trains multiple decision trees, each on a distinct sample of the initial training dataset achieved through resampling, and then combines their predictions to derive the outcome. This process is known as bagging (bootstrap aggregation). Random Forest is popular for delivering accurate predictions and handle datasets with a large number of features efficiently.

In [ ]:
# setting the parameters for the model
model_rf = RandomForestClassifier(n_estimators=500 , oob_score = True, n_jobs = -1,
                                  random_state =50,max_leaf_nodes = 30)
# fitting the model
model_rf.fit(X_train, y_train)

# make predictions
prediction_test = model_rf.predict(X_test)

In [ ]:
# measure the model performance
model_performance(y_test,prediction_test, 'Random Forest')

In [ ]:
cv_results = cross_validate(model_rf, X_train, y_train, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy for Random Forests Model is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The accuracy of the model is very close to 80%, however, the mean cross-validation accuracy is higher than the logistic regression (LR) model. The model has yielded less number of False positives compared to the LR model. However, it is possible that we can further improve this model by tuning the hyperparameter and getting more accurate results. 

Next I will move to the survval analysis models.

## Survival Analysis

The machine learning models we've used to predict customer churn can inform us about which customers are likely to churn, but they don't provide insight into when this event will occur. In contrast, Survival Analysis offers a powerful solution by not only predicting the likelihood of customers discontinuing their business but also estimating when that event is likely to happen. 


Survival Analysis contains a range of statistical methods designed to model the time until an event occurs, such as how long it takes for customers to churn. These models are useful when dealing with censored data, where some training data only provides partial observations. In our dataset, some customers have not yet churned within the observed time frame, making their future churn times uncertain. This type of data is referred to as censored, as it lacks complete information beyond the observation point.

Survival models rely on a survival prediction function, which can take the form of either a survival function or a cumulative hazard function. The survival function quantifies the likelihood that the event of interest (e.g., customer churn) has not happened up to a given time 't,' essentially representing the probability of survival beyond that time point. On the other hand, the hazard function provides insights into the event's occurrence rate or intensity, shedding light on when and how the event is likely to happen.

There are a few survival models such as the COX semi-parametric model, Weibull (with and without covariates), and Survival Forest. I am only exploring a few, popular models here.

For this analysis, I will use the **lifelines** and **scikit-survival** libraries in Python. Alternatively, you can also use the **PySurvival** library. 

In [ ]:
# install the lifeline library
! pip install lifelines

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter

In [ ]:
# selecting some features of the initial dataset to apply the Kaplan-Meier estimator
input_data = df[['SeniorCitizen', 'Partner', 'Dependents', 'PaperlessBilling','MonthlyCharges', 'Contract','tenure', 'Churn', 'PaymentMethod']]
input_data.head(3)

### Kaplan-Meier estimator

The Kaplan-Meier estimator is a non-parametric statistical tool that estimates the survival function. It constructs a survival curve using a stepwise approach, with each step corresponding to specific time points at which one or more customers experienced the event of interest, like churn. Kaplan-Meier estimator is not suitable for making predictions into the future since it simply extends the last observed survival probability. However, it is useful for understanding the overall population data and characterizing the survival patterns.

* T:  the time of event of interest
* E: (Censoring)  binary event indicator, determine between censored (0) and uncensored (1) observations.

In [ ]:
# select T and E from the input data
T = input_data['tenure']
E = input_data['Churn']

#fit them to the kmf model
kmf = KaplanMeierFitter()
kmf.fit(T, E)

kmf.survival_function_
kmf.cumulative_density_
kmf.plot_survival_function()

The survival curve of the KM estimate represents each time point and  the probability of surviving the event (e.g., Churn event). For instance, around 40 months, there is a 70% probability that the customers will remain with the company, and there is a 30% probability that they leave. 

The shaded region around the KM estimate curve represents the confidence interval, which tells how certain we can be that the true probability is within the shaded region.

KM estimate is useful for gaining insights into the subgroups within the population. 

In [ ]:
# KM estimates based on the contract type
ax = plt.subplot(111)

for name, grouped_df in input_data.groupby('Contract'):
    kmf.fit(grouped_df["tenure"], grouped_df["Churn"], label=name)
    kmf.plot_survival_function(ax=ax)

This plot indicates a notable trend: customers with month-to-month contracts shows a greater likelihood of churning compared to those with one or two-year contracts, aligning with real-world observations.

In [ ]:
# KM estimates based on the contract type
ax = plt.subplot(111)

for name, grouped_df in input_data.groupby('PaymentMethod'):
    kmf.fit(grouped_df["tenure"], grouped_df["Churn"], label=name)
    kmf.plot_survival_function(ax=ax)

Similarly, customers who use electronic checks to pay the bills have a higher probability of churning than customers who make automatic payments. 

Now let's move to more advanced methods in survival analysis.

###  Cox’s proportional hazard’s model

Cox's Proportional Hazards Model is a widely used survival regression model that estimates the influence of individual variables on survival outcomes. It represents the hazard rate as a function of time and various covariates. Hence, enables us to quantify how each predictor affects the risk of an event occurring over time while maintaining the assumption that hazard ratios remain proportional. 

In [ ]:
# let's use the same set of independent variables we used before with the ML model
X_tr, X_te, y_tr, y_te = train_test_split(X, y, stratify = y, test_size=0.2, random_state=33)

In [ ]:
# adding back the target variable
X_tr.loc[:,'Churn'] = y_tr

In [ ]:
# fitting the Cox's Proportional
cph = CoxPHFitter()
cph.fit(X_tr, duration_col='tenure', event_col='Churn')

# print the model summary
cph.print_summary() 

Now let's try to inerpret the model summary by using some of the above information.

* **p-value** : The p-value associated with each coefficient assesses the statistical significance of that variable's impact on survival. Lower p-values (typically less than 0.05) suggest that the variable is likely to be statistically significant, indicating that it has a meaningful influence on survival. 

--According to the above summary 'contract', 'PaperlessBilling', and 'Partner' has lower-

* **Hazard Ratio (exp(coef))**: The hazard ratio is derived from the exponentiated coefficients (exp(coef)). It tells you how a one-unit change in the predictor affects the hazard rate. A hazard ratio greater than 1 implies an increased hazard, while a hazard ratio less than 1 suggests a decreased hazard. For example, a hazard ratio of 2 means that the hazard doubles with each one-unit increase in the predictor.

* **Concordance Index (C-index)**: The C-index is a measure of the model's predictive accuracy. It assesses how well the model discriminates between individuals who experienced the event and those who did not. A C-index of 0.5 represents random chance, while higher values (closer to 1) indicate better predictive performance. Since our model achive 0.82 C-Index, we can say that it fairly performs well. 

In [ ]:
cph.plot()

The cph plot assesses whether the hazard ratios for the covariate(s) being examined remain relatively constant over time. If the lines or curves for different levels of the covariate stay close to the reference line (HR = 1), it suggests that the proportional hazards assumption is met. However, if the lines cross or diverge significantly, it indicates a violation of the assumption, implying that the covariate's effect on the hazard rate varies over time.

From the above plot, we can see that 'Contract_one year', Contract_two year, and 'Partner' coeeficients are aways from the reference line. Another way to check the deviations of the hazards assumption is to use the check_assumptions method.

```python
cph.check_assumptions(X_tr, show_plots=True, p_value_threshold=0.05)
```
There are various methods to penalize Cox models and improve the model. I will not be exploring those here. For more details, see : https://scikit-survival.readthedocs.io/en/stable/user_guide/coxnet.html 

Next, let's look at the prediction results of our Cox's Proportional Hazards Model.


In [ ]:
# add the churn to testing features
X_te.loc[:,'Churn'] = y_te

# select the customers who has not churned yet for predicitng the probabilities
X_sel=X_te.loc[X_te['Churn']==0]

In [ ]:
# predict the survival function for individuals, given their covariates.
cph.predict_survival_function(X_sel)
# return the predicted median survival time for each individual in your dataset.
cph.predict_median(X_sel)
# returns the partial hazard for the individuals, partial since the baseline hazard is not included
cph.predict_partial_hazard(X_sel)

In [ ]:
# compute the expected lifetime, 𝐸[𝑇] using covariates X
X_pred=X_sel.copy()
X_pred.loc[:,'Predicted tenure'] = cph.predict_expectation(X_sel)

In [ ]:
# sample of a predictions
X_pred.head(10)

In [ ]:
# sample of the survival curves, based on the probability predictions
# x-axis shows the time while y-axis shows the probabilities
cph.predict_survival_function(X_sel.head(10)).plot()

Now, I want to look at some of the individual prediction results and combine the insights we got from the KM estimate.

In [ ]:
X_pred.loc[(X_pred['Contract_One year'] == 0) & (X_pred['Contract_Two year'] == 0)].head()

In [ ]:
X_pred.loc[(X_pred['Contract_One year'] == 1) | (X_pred['Contract_Two year'] == 1)].head()

The observations indicate the following:
1. Customers with one-year or two-year contracts show higher probabilities, suggesting longer tenures (lifetime) with the company.
2. Customers on monthly plans have lower probabilities, suggesting shorter lifetimes with the company.

These findings align with the survival function estimates we derived for the overall population using the Kaplan-Meier estimator.

### Random Survival Forests

The Random Survival Forest is similar to the Random Forest ensemble approach and utilises tree-based learners. As previously mentioned, it builds upon choosing different bootstrap samples to create a different tree from the original data. In each decision node, a subset of features is chosen at random to assess the splitting criteria and thresholds. The final prediction is derived by aggregating the predictions from all the individual trees in the ensemble. 

Scikit-survival is a Python library for survival analysis that extends scikit-learn to include survival analysis algorithms and tools. First, let's install the library.

In [ ]:
# install scikit-survival library
!pip install scikit-survival

In [ ]:
#from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv

In [ ]:
# I am using the same data set used before with the selected features
all_features_df.head(3)

In [ ]:
# seperating training and testing data, bit different from the previous since we drop both Churn (E) and Tenure (T) from X
X = all_features_df.drop(['Churn','tenure'],axis=1)
y = all_features_df[['Churn', 'tenure']]

random_state = 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

In [ ]:
# the model requires both T and E values as input to y
y_train.head(3)

In [ ]:
# create structured array from data frame, we need to convert y dataframe into structured array
new_y_arr = Surv.from_dataframe('Churn', 'tenure', y_train)
new_y_arr

In [ ]:
# set the aparameters for the model
random_state = 20
rsf = RandomSurvivalForest(
    n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=random_state
)
# fit the model
rsf.fit(X_train, new_y_arr)

We can check how well the model performs by evaluating it on the test data. 

In [ ]:
# format the testing y data
y_test_arr = Surv.from_dataframe('Churn', 'tenure', y_test)

# measure the C-Index 
rsf.score(X_test, y_test_arr)

The **concordance index (CI)** is closer to 0.82, which is a good value. The survival regression model based on Cox's Proportional Hazards and the survival forest have similar CIs.  


When making predictions, a new sample is run down the tree until it reaches a terminal node. Each terminal node non-parametrically estimates the survival and cumulative hazard function using the Kaplan-Meier and Nelson-Aalen estimators, respectively. The method also calculates a risk score that measures the expected number of events for one particular terminal node.

In [ ]:
y_test.reset_index().drop(['index'], axis=1, inplace= True)

In [ ]:
# generate predictions of the risk scores
y_test['pred_risk_scores']=rsf.predict(X_test)
# concat predictions and the actual values
y_test.loc[y_test['Churn']==0].head()

Now let us look at the predicted survival probabilities based on the characteristics of each customer. Todo so, I am going to combine the features and the T, E and the predicted risk scores. 

In [ ]:
# reset index
X_test.reset_index().drop(['index'], axis=1, inplace= True)
# concat the testing data
X_all = pd.concat([X_test, y_test], axis=1)

In [ ]:
monthly_contr =X_all.loc[(X_all['Contract_One year'] == 0) & (X_all['Contract_Two year'] == 0) & (X_all['Churn'] == 0)]
monthly_contr.head()

In [ ]:
yearly_contr = X_all.loc[(X_all['Contract_One year'] == 1) | (X_all['Contract_Two year'] == 1) & (X_all['Churn'] == 0)]
yearly_contr.head()

Consistent with the outcomes from our survival regression and Kaplan-Meier estimation, we observe high risk scores among customers with month-to-month contracts, indicating a higher likelihood of churn. Conversely, customers committed to one or two-year contracts show lower risk scores, suggesting a lower probability of churn over the given time period.

We can gain more insights by looking at the predicted survival function of the customer data.

In [ ]:
# seperate monthly and yearly contract of test features
X_test_monthly = monthly_contr[X_test.columns]
X_test_yearly = yearly_contr[X_test.columns]

In [ ]:
# generate the Survival Probability plot
def survival_probability_plot(surv, contract):
    for i, s in enumerate(surv):
        plt.step(rsf.unique_times_, s, where="post", label=str(i))
    plt.ylabel("Survival probability")
    plt.xlabel("Time in Months")
    plt.title('Survival probabilities of cutomers on: ' + contract)
    plt.legend()
    plt.grid(True)

In [ ]:
# select a sample of the test data (only customers on monthly contract) to visulize the survival probabilities
surv_monthly_contract = rsf.predict_survival_function(X_test_monthly.sample(n=5), return_array=True)
survival_probability_plot(surv_monthly_contract, 'Month-to-month')

In [ ]:
# select a sample of the test data to visulize the survival probabilities
surv_monthly_contract = rsf.predict_survival_function(X_test_yearly.sample(n=5), return_array=True)
survival_probability_plot(surv_monthly_contract, "One or Two years")

The survival probability plot shows the estimated survival probability and the time points at which survival probabilities are estimated. Consistent, with our previous conclusions, customers who are on a one-year or two-year contract has higher survival probabilities than for the customers on monthly contract from churning.

#### Permutation based feature importance

Permutation feature importance is a model inspection technique. The scikit-learn toolbox provides a method to determine the permutation feature importance of the features used in the survival forest model. See https://scikit-learn.org/stable/modules/permutation_importance.html for more details.

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
# measure the feature importance 
result = permutation_importance(rsf, X_test, y_test_arr, n_repeats=15, random_state=random_state)

In [ ]:
# meand and the std values of the feature importance
pd.DataFrame(
    {
        k: result[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X_test.columns,
).sort_values(by="importances_mean", ascending=False)

The results show that the most important features of the model are the contract type and the monthly charges. There is still room for this model. I have not done any hyperparameter tuning with the model, and it may be a good starting point to improve the model further.

### Conclusions

* I have applied both machine learning and survival analysis techniques to tackle the customer churn prediction challenge. While all the models have indicated good performance, ensemble methods like random forests and survival random forests offer better accuracy and efficiency.
* ML models primarily focus on identifying which customers are likely to churn within a given pool of customers. 
* In contrast, survival analysis addresses the question of when a customer is likely to churn, offering probabilities of customer survival over specific time periods.
* The choice between these two approaches depends on the nature of the customer churn issue a company is dealing with. By understanding "who" is likely to churn through ML or "when" they are likely to churn through survival analysis, companies can change or modify their retention strategies more effectively and take proactive measures to extend their customers' relationships.